In [ ]:
import open3d as o3d 
import numpy as np
from plyfile import *
import scipy.stats
import random
import pandas as pd

In [ ]:
##############导入点云数据#################
pcd_AC13_1 = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/AC13-1.ply")
pcd_AC13_2 = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/AC13-2.ply")
pcd_AC16_3 = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/AC16-3.ply")
pcd_AC16_4 = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/AC16-4.ply")
pcd_UT5_1  = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/UT5-1.ply")
pcd_UT5_2  = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/UT5-2.ply")
pcd_UT5_3  = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/UT5-3.ply")
pcd_UT5_4  = o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/UT5-4.ply")
pcd_SMA13_1= o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/SMA13-1.ply")
pcd_SMA13_2= o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/SMA13-2.ply")
pcd_OGFC10_1= o3d.io.read_point_cloud("C:/Users/Administrator/Desktop/surface/OGFC10-1.ply")

#############点云标准化####################
def standard_pointcloud(point_cloud):
    Point_standard=[]
    Point=np.asarray(point_cloud.points)    
    x=np.array(Point[:,0])[...,np.newaxis]
    y=np.array(Point[:,1])[...,np.newaxis]
    z=np.array(Point[:,2])[...,np.newaxis]
    Point_standard=np.concatenate((x-np.min(x),y-np.min(y),z-np.min(z)),axis=-1)
    return Point_standard
    
AC13_1=standard_pointcloud(pcd_AC13_1)
AC13_2=standard_pointcloud(pcd_AC13_2)
AC16_3=standard_pointcloud(pcd_AC16_3)
AC16_4=standard_pointcloud(pcd_AC16_4)
UT5_1=standard_pointcloud(pcd_UT5_1)
UT5_2=standard_pointcloud(pcd_UT5_2)
UT5_3=standard_pointcloud(pcd_UT5_3)
UT5_4=standard_pointcloud(pcd_UT5_4)
SMA13_1= standard_pointcloud(pcd_SMA13_1)
SMA13_2= standard_pointcloud(pcd_SMA13_2)
OGFC10_1 =  standard_pointcloud(pcd_OGFC10_1)


point_cloud_array_group = []
point_cloud_array_group.append(AC13_1)
point_cloud_array_group.append(AC13_2)
point_cloud_array_group.append(AC16_3)
point_cloud_array_group.append(AC16_4)
point_cloud_array_group.append(UT5_1)
point_cloud_array_group.append(UT5_2)
point_cloud_array_group.append(UT5_3)
point_cloud_array_group.append(UT5_4)
point_cloud_array_group.append(SMA13_1)
point_cloud_array_group.append(SMA13_2)
point_cloud_array_group.append(OGFC10_1)

In [ ]:


###############点云空间分布##############
def Point_distribution(point_cloud_array, num_classify=100, range_uplimt=10):
    step=range_uplimt/num_classify
    Index_list=[(i*step) for i in range(num_classify)]
    result = np.zeros(np.array(Index_list).shape)
    for i in point_cloud_array:
        for j in range(len(Index_list)):
            if i[-1]<Index_list[j]:
                result[j]+=1
                break
    #计算古典概率
    for i in range(len(result)):
        result[i]=result[i]/len(point_cloud_array)
    return result

##############JS散度计算##############
def JS_divergence(p,q):
    M=(p+q)/2
    return 0.5*scipy.stats.entropy(p, M, base=2)+0.5*scipy.stats.entropy(q, M, base=2)

#########根据JS散度阈值确定切割尺度##############
def JS_threshold(point_cloud_array_origion,Thr =0.025, min_size = 30,seed_num= 20):
    L=np.max(point_cloud_array_origion[:,0])
    W=np.max(point_cloud_array_origion[:,1])
    H=np.max(point_cloud_array_origion[:,2])
    max_size=int(min(L,W))
    h_new = min_size
    History=[]
    for i in range(max_size,min_size,-5):
        ###计算布种区域(正方形)
        new_W_min=int(i/2)
        new_W_max=W-int(i/2)
        new_L_min=int(i/2)
        new_L_max=L-int(i/2)
        ###随机布种,种数为seed_num= 20,正方形中点坐标
        res_W= [random.uniform(new_W_min,new_W_max) for _ in range(seed_num)]
        res_L= [random.uniform(new_L_min,new_L_max) for _ in range(seed_num)]
        for j in range(seed_num):
            JS_value=np.zeros(seed_num)
            ##根据布种切分出第j个小区域
            temp_point_cloud = point_cloud_array_origion[np.where((point_cloud_array_origion[:,1]<(res_W[j]+i/2))&
                                                                  (point_cloud_array_origion[:,1]>(res_W[j]-i/2))&
                                                                  (point_cloud_array_origion[:,0]>(res_L[j]-i/2))&
                                                                  (point_cloud_array_origion[:,0]<(res_L[j]+i/2)))]
            #print(np.array(temp_point_cloud).shape)
            #####计算第j个小区域与原始图像的JS散度
            JS_value[j] = JS_divergence(Point_distribution(temp_point_cloud),Point_distribution(point_cloud_array_origion))
            #print(JS_value)
        ####计算最大JS，并比较阈值判断是否继续
        JS_now=np.max(JS_value)
        History.append(JS_now)
        print(JS_now)
        if JS_now>Thr:
            h_new = i+5
            break
    return h_new, History

##########根据11张图的最大切割尺度确定最终切割尺度#################
h_final=np.zeros(11)
History=[]
for i in range(11):
    h_final[i],History_temp=JS_threshold(point_cloud_array_group[i])
    History.append(History_temp)
H_final =np.max(h_final)
print(H_final)

In [ ]:
#np.savetxt("C:/Users/Administrator/Desktop/surface/实验结果存放/切分JS.txt",History,fmt='%s')
history=pd.DataFrame(History)
writer = pd.ExcelWriter('C:/Users/Administrator/Desktop/surface/实验结果存放/切分JS_new.xlsx')  #关键2，创建名称为hhh的excel表格
history.to_excel(writer,'page_1',float_format='%.5f')  #关键3，float_format 控制精度，将data_df写到hhh表格的第一页中。若多个文件，可以在page_2中写入
writer.save()  

In [ ]:
################绘制切分后的点云小图并查看#####################
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(AC13_1)
o3d.visualization.draw_geometries([pcd])

In [ ]:
######################按照计算的H_final进行窗口滑动切分#########################
H_final=51
AC13=[0,6,4,3.5,5.5,7.5,10.5,16,23.5,18.5,5,0]#级配
AC16=[0,5.4,2.4,2.4,3.7,4.1,7.8,5.5,26.3,20.1,16.6,5.7]#级配
SMA13=[0,10.7,2.1,1.4,1.1,1.2,2.9,9.4,30.3,31.5,8.2,1.2]#级配
UT5=[0,6.7,2,2.5,5,20,13.4,48.8,1.5,0.1,0,0]#级配
def split_slide(point_cloud_array_origion,H_new=H_final, intervial =[5,5]):
    L=int(np.max(point_cloud_array_origion[:,0]))
    W=int(np.max(point_cloud_array_origion[:,1]))
    Point_cloud_small = []
    L_intervial = intervial[0]
    W_intervial = intervial[1]
    for i in range (0,L-H_new,L_intervial):
        for j in range (0,W-H_new,W_intervial):
            temp_point_cloud = point_cloud_array_origion[np.where((point_cloud_array_origion[:,1]>i)&
                                                                  (point_cloud_array_origion[:,1]<(H_new+i))&
                                                                  (point_cloud_array_origion[:,0]>j)&
                                                                  (point_cloud_array_origion[:,0]<(H_new+j)))]
            Point_cloud_small.append(temp_point_cloud)
    return Point_cloud_small
####################原始图像拆分子点云集########################################
PointCloud_new =[]
for i in range(11):
    PointCloud_new=np.concatenate((PointCloud_new,split_slide(point_cloud_array_group[i])),axis=0)
print(len(PointCloud_new))

In [ ]:
################绘制切分后的点云小图并查看#####################
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(PointCloud_new[1])
o3d.visualization.draw_geometries([pcd])

In [ ]:
######################按照计算的H_final进行窗口滑动切分#########################
H_final=51
AC13=[0,6,4,3.5,5.5,7.5,10.5,16,23.5,18.5,5,0]#级配
AC16=[0,5.4,2.4,2.4,3.7,4.1,7.8,5.5,26.3,20.1,16.6,5.7]#级配
SMA13=[0,10.7,2.1,1.4,1.1,1.2,2.9,9.4,30.3,31.5,8.2,1.2]#级配
UT5=[0,6.7,2,2.5,5,20,13.4,48.8,1.5,0.1,0,0]#级配
def split_slide(point_cloud_array_origion,H_new=H_final, intervial =[5,5]):
    L=int(np.max(point_cloud_array_origion[:,0]))
    W=int(np.max(point_cloud_array_origion[:,1]))
    Point_cloud_small = []
    L_intervial = intervial[0]
    W_intervial = intervial[1]
    for i in range (0,L-H_new,L_intervial):
        for j in range (0,W-H_new,W_intervial):
            temp_point_cloud = point_cloud_array_origion[np.where((point_cloud_array_origion[:,1]>i)&
                                                                  (point_cloud_array_origion[:,1]<(H_new+i))&
                                                                  (point_cloud_array_origion[:,0]>j)&
                                                                  (point_cloud_array_origion[:,0]<(H_new+j)))]
            Point_cloud_small.append(temp_point_cloud)
    return Point_cloud_small
####################原始图像拆分子点云集########################################
PointCloud_new =[]
for i in range(10):
    PointCloud_new=np.concatenate((PointCloud_new,split_slide(point_cloud_array_group[i])),axis=0)
print(len(PointCloud_new))
#计算最小子点云点数
num_min=PointCloud_new[0].shape[0]
for i in range(len(PointCloud_new)):
    num_point = PointCloud_new[i].shape[0]
    if num_min>num_point:
        num_min = num_point
print(num_min)
#根据最小点云数进行降采样，根据设置的点数降采样
def pointCloud_downsampling(Point_cloud_array, num_pointCloud=30000):
    # estimate radius for rolling ball
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(Point_cloud_array)
    pcd.estimate_normals()
    # estimate radius for rolling ball
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 1.5 * avg_dist   
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))
    pcd_new = o3d.geometry.TriangleMesh.sample_points_uniformly(mesh, number_of_points=num_pointCloud) # 采样点云
    #o3d.visualization.draw_geometries([pcd_new],width=1024, height=1024)
    New_Point_cloud_array = np.asarray(pcd_new.points)
    return New_Point_cloud_array

PointCloud_Standard =[]
for i in range(len(PointCloud_new)):
    PointCloud_Standard.append(pointCloud_downsampling(PointCloud_new[i],30000))


In [ ]:
################绘制切分后的点云小图并查看#####################
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(PointCloud_Standard[250])
o3d.visualization.draw_geometries([pcd])

In [ ]:
########绘制示意图################
x1 = Point_distribution(AC16_3)
###h=50切分
AC16_3_50 = AC16_3[np.where((AC16_3[:,1]>20)&
                            (AC16_3[:,1]<70)&
                            (AC16_3[:,0]>10)&
                            (AC16_3[:,0]<60))]
x2 = Point_distribution(AC16_3_50)
#####h=15切分######
AC16_3_15 = AC16_3[np.where((AC16_3[:,1]>20)&
                            (AC16_3[:,1]<35)&
                            (AC16_3[:,0]>10)&
                            (AC16_3[:,0]<25))]
x3 = Point_distribution(AC16_3_15)
"""
pcd1=o3d.geometry.PointCloud()
pcd1.points=o3d.utility.Vector3dVector(AC16_3_15)

####上色#########
colors = np.zeros([AC16_3_15.shape[0], 3])
height_max = 10
height_min = 0
delta_c = abs(height_max - height_min) / (255)
for j in range(AC16_3_15.shape[0]):
    color_n = (AC16_3_15[j, 2] - height_min) * delta_c
    colors[j, :] = [color_n, 1 - 2*color_n, color_n]      
pcd1.colors = o3d.utility.Vector3dVector(colors)  
####上色#########
o3d.visualization.draw_geometries([pcd1],width=1024, height=1024)
"""
x=np.concatenate([x1[np.newaxis,...],x2[np.newaxis,...],x3[np.newaxis,...]],axis=0)
X=pd.DataFrame(x)
writer = pd.ExcelWriter('C:/Users/Administrator/Desktop/surface/实验结果存放/切分示意图.xlsx')  #关键2，创建名称为hhh的excel表格
X.to_excel(writer,'page_1',float_format='%.5f')  #关键3，float_format 控制精度，将data_df写到hhh表格的第一页中。若多个文件，可以在page_2中写入
writer.save()  